In [3]:
!pip install torch==1.6.0
!pip install opencv-python
!pip install torchvision==0.2.2
!pip install albumentations
!pip install tensorflow
!pip install pytorch-lightning

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 748.8 MB 16 kB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 49.5 MB 29.4 MB/s 
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 64 kB 5.6 MB/s 
     |████████████████████████████████| 52 kB 3.7 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user i

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import typing as tp
import yaml
import random
import os
import sys
import soundfile as sf
import librosa
import cv2
import matplotlib.pyplot as plt
import time
import glob
from tqdm import tqdm

import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# import resnest.torch as resnest_torch

from torchvision import models

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
# from resnet import ResNet, Bottleneck

from albumentations.core.transforms_interface import DualTransform, BasicTransform
import albumentations as albu

from sklearn.model_selection import StratifiedKFold

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

## util

In [2]:
config_set = {
    'dataset': {
          'name': 'SpectrogramDataset',
          'params': {
            'img_size': 224, 
            'melspectrogram_parameters': {
                'n_fft': 1024,
                'hop_length': 320,
                'win_length': 1024,
                'n_mels': 64, 
                'fmin': 50, 
                'fmax': 24000, 
            }
      }
    },
    'loader': {
      'train': {
        'batch_size': 6,
        'shuffle': True,
        'num_workers': 2,
        'pin_memory': True,
        'drop_last': True,
      },
      'valid': {
        'batch_size': 2,
        'shuffle': False,
        'num_workers': 2,
        'pin_memory': True,
        'drop_last': True,
      }
    }
}
SEED=1213
PERIOD = 10
SPECIES_NUM = 24
EPOCH = 50
HOP_LEN = 512
SR = 48000
BATCH = 2

In [3]:
config = config_set

In [4]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
set_seed(SEED)

In [5]:
INPUT_ROOT = Path("/home/knikaido/work/Rainforest-Connection/data")
RAW_DATA = INPUT_ROOT / "rfcx-species-audio-detection"
TRAIN_AUDIO_DIR = RAW_DATA / "train"
# TRAIN_RESAMPLED_AUDIO_DIRS = [
#   INPUT_ROOT / "birdsong-resampled-train-audio-{:0>2}".format(i)  for i in range(5)
# ]
TEST_AUDIO_DIR = RAW_DATA / "test"
OUTPUT_DIR = './output/'

In [6]:
train_gby = pd.read_pickle(RAW_DATA / "train_gby_mel.pkl")
train_gby.head()

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max,name
0,003bec244,[14],[1],[44.544],[2531.25],[45.1307],[5531.25],/home/knikaido/work/Rainforest-Connection/Git/...
1,006ab765f,[23],[1],[39.9615],[7235.16],[46.0452],[11283.4],/home/knikaido/work/Rainforest-Connection/Git/...
2,007f87ba2,[12],[1],[39.135999999999996],[562.5],[42.272],[3281.25],/home/knikaido/work/Rainforest-Connection/Git/...
3,0099c367b,[17],[4],[51.4206],[1464.26],[55.1996],[4565.04],/home/knikaido/work/Rainforest-Connection/Git/...
4,009b760e6,[10],[1],[50.0854],[947.461],[52.5293],[10852.7],/home/knikaido/work/Rainforest-Connection/Git/...


In [7]:
def mono_to_color(
    X: np.ndarray, mean=None, std=None,
    norm_max=None, norm_min=None, eps=1e-6
):
    # Stack X as [X,X,X]
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    X = X - mean
    std = std or X.std()
    Xstd = X / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Normalize to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

In [8]:
def get_criterion():
    pos_weights = torch.ones(SPECIES_NUM)
    pos_weights = pos_weights * SPECIES_NUM
    loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
    return loss_function

In [9]:
def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.

    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    """Pad framewise_output to the same length as input frames. The pad value
    is the same as the value of the last frame.
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1:, :].repeat(
        1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output

def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
def init_bn(bn):
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.0)


class AttBlock(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear",
                 temperature=1.0):
        super().__init__()

        self.activation = activation
        self.temperature = temperature
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.bn_att = nn.BatchNorm1d(out_features)
        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)
        init_bn(self.bn_att)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)

In [10]:
class LitModule(pl.LightningModule):
    
    def __init__(self, train_len, mel_params, num_classes=SPECIES_NUM):
        super().__init__()
        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None
        self.interpolate_ratio = 30  # Downsampled ratio
        # load pretrained models, using ResNeSt-50 as an example
        base_model = torch.hub.load('pytorch/vision:v0.6.0', 'densenet121', pretrained=True)
        layers = list(base_model.children())[:-1]
        self.encoder = nn.Sequential(*layers)

        in_features = 1024

        self.fc1 = nn.Linear(in_features, in_features, bias=True)
        self.att_block = AttBlock(in_features, num_classes, activation="sigmoid")
        
        
        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=mel_params['n_fft'], hop_length=mel_params['hop_length'], 
            win_length=mel_params['win_length'], window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(sr=SR, n_fft=mel_params['n_fft'], 
            n_mels=mel_params['n_mels'], fmin=mel_params['fmin'], fmax=mel_params['fmax'], ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(mel_params['n_mels'])
        self.init_weight()

        self.init_weight()
#         self.criterion = bce_criterion
        self.train_len = train_len
        
    def init_weight(self):
        init_layer(self.fc1)
        init_bn(self.bn0)

    def forward(self, x, istrain=True, mixup_lambda=None):
        
#         if self.training and mixup_lambda is not None:
#             x = do_mixup(x, mixup_lambda)
        
        x = self.spectrogram_extractor(x)
        # batch_size x 1 x time_steps x freq_bins
        x = self.logmel_extractor(x)
        # batch_size x 1 x time_steps x mel_bins

        frames_num = x.shape[2]

        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        #print(x.shape)
        
#         if istrain:
#             x = self.spec_augmenter(x)
#         if istrain and mixup_lambda is not None  :
#             x = do_mixup(x, mixup_lambda)

        # Output shape (batch size, channels, time, frequency)
        x = x.expand(x.shape[0], 3, x.shape[2], x.shape[3])
        
        x = self.encoder(x)

        # Aggregate in frequency axis
        x = torch.mean(x, dim=3)

        # channel smoothing
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2

        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)
        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        # Get framewise output
        framewise_output = interpolate(segmentwise_output,
                                       self.interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)

        output_dict = {
            "framewise_output": framewise_output,
            "logit": logit,
            "clipwise_output": clipwise_output
        }

        return output_dict
    
    def configure_optimizers(self):
#         optimizer = torch.optim.SGD(model.parameters(), lr=0.001, weight_decay=0.01, momentum=0.9)
        optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=True)
#         scheduler = {'scheduler': optim.lr_scheduler.OneCycleLR(
#                                                 optimizer,
#                                                 max_lr=self.learning_rate,
#                                                 steps_per_epoch=int(len(self.train_dataloader())),
#                                                 epochs=self.hparams.epochs,
#                                                 anneal_strategy="linear",
#                                                 final_div_factor = 30,
#                                             ),
#                                 'name': 'learning_rate',
#                                 'interval':'step',
#                                 'frequency': 1}
        scheduler = {'scheduler': torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=0.001*0.01),
                                'name': 'learning_rate',
                                'interval':'epoch',
                                'frequency': 1}
#         scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
#         lr_scheduler = {"scheduler": scheduler }
        return [optimizer], [scheduler]
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        x, y_a, y_b, lam = mixup_data(x, y, alpha=0.5)
#         mixup_lambda = self.mixup_augmenter.get_lambda(6).to("cuda:0")
#         y = do_mixup(y, mixup_lambda)
        y_pred = self.forward(x, istrain=True)
        loss = mixup_criterion(self.criterion, y_pred['clipwise_output'], y_a, y_b, lam)
#         print(y_pred['clipwise_output'].shape, y.shape)
#         loss = self.criterion(y_pred['clipwise_output'], y)
        self.log('train_loss', loss,  on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        y_pred = self.forward(x, istrain=False)
#         print(y_pred)
        loss_max = self.criterion(torch.max(y_pred['framewise_output'], 1)[0], y)
        loss_clip = self.criterion(y_pred['clipwise_output'], y)
        loss = loss_clip + 0.5*loss_max
        y_pred_act = y_pred['clipwise_output']
        lwlap_step, weight_step = lwlap_wrapper(y, y_pred_act)
        lwlap_step = (lwlap_step * weight_step).sum()
#         print('valid_epoch_loss = ', loss)
#         print('valid_epoch_loss = ', lwlap_step)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('lwlap_score', lwlap_step, on_epoch=True, prog_bar=True, logger=True)
        return loss, lwlap_step

    def validation_epoch_end(self, validation_step_outputs):
        validation_step_outputs = np.array(validation_step_outputs)
        validation_step_losses = validation_step_outputs[:, 0]
        mean_loss = torch.stack([x for x in validation_step_losses]).mean()
        
        validation_step_scores = validation_step_outputs[:, 1]
        mean_score = torch.stack([x for x in validation_step_scores]).mean()

        print('valid_epoch_loss = ', mean_loss)
        print('valid_epoch_lwlap = ', mean_score)
        self.log('valid_epoch_loss', mean_loss, prog_bar=True, logger=True)
        self.log('valid_epoch_lwlap', mean_score, prog_bar=True, logger=True)
#         tqdm.write('Dice: \t%.3f' % mean_loss)
        return mean_loss, mean_score

In [11]:
model = LitModule(0, config['dataset']['params']['melspectrogram_parameters'])

Using cache found in /home/user/.cache/torch/hub/pytorch_vision_v0.6.0


In [12]:
model.load_state_dict(torch.load(OUTPUT_DIR + 'model'))

<All keys matched successfully>

In [13]:
def signal_to_mel(y, sr, mel_params):
    
    len_y = len(y)
    effective_length = int(SR * PERIOD)
    
    start = 0
    end = start + effective_length
    
    images = []
    while(start < len_y):
        if(end > len_y):
            break
        y_ele = y[start:end]
        
#         melspec = librosa.feature.melspectrogram(y_ele, sr=sr, **mel_params['melspectrogram_parameters'])
#         melspec = librosa.power_to_db(melspec).astype(np.float32)
    
#         image = mono_to_color(melspec)
#         height, width, _ = image.shape
# #         image = cv2.resize(image, (int(width * mel_params['img_size'] / height), mel_params['img_size']))
#         image = np.moveaxis(image, 2, 0)
#         image = (image / 255.0).astype(np.float32)
#         image = torch.from_numpy(y_ele).clone()
        images.append(y_ele.astype(np.float32))
        
        start = end
        end += effective_length
                
    return np.array(images)

In [14]:
test_wav_pathes = sorted(glob.glob(str(TEST_AUDIO_DIR / '*.flac')))
len(test_wav_pathes)

1992

In [15]:
device = torch.device("cuda")
model.eval().to(device)
preds = []

for path in tqdm(test_wav_pathes):
    y, sr = sf.read(path)
    mel_img = signal_to_mel(y, sr, config["dataset"]["params"])
#     mel_img = torch.from_numpy(mel_img).clone().to(device)
#     pred = model(mel_img)
# #         pred = torch.mean(pred['clipwise_output'], 0)
# #         pred = pred.to('cpu').detach().numpy().copy()
#     preds.append(pred)
    
    
    b_start = 0
    b_end = BATCH
    b_preds = []
    while(b_start < mel_img.shape[0]):
        if(b_end >= mel_img.shape[0]):
            mel_batch = mel_img[b_start:]
        else:
            mel_batch = mel_img[b_start:b_end]
        mel_batch = torch.from_numpy(mel_batch).clone().to(device)
        pred = model(mel_batch, istrain=False)
        pred = pred['clipwise_output'].to('cpu').detach().numpy().copy()
        b_preds.extend(pred)
        b_start += BATCH
        b_end += BATCH
    pred = np.mean(b_preds, axis=0)
    preds.append(pred)    
            
preds = np.array(preds)
#     break

100%|██████████| 1992/1992 [02:25<00:00, 13.73it/s]


In [16]:
sub = pd.read_csv(str(RAW_DATA / 'sample_submission.csv'))

In [17]:
sub.loc[:, 's0':'s23'] = preds

In [18]:
sub.to_csv(OUTPUT_DIR + '95_sub.csv', index=False)

In [19]:
sub

,recording_id,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
0,000316da7,0.000027,3.506493e-06,3.985078e-05,0.242470,3.717819e-05,0.001449,3.641636e-02,7.482922e-05,8.370019e-05,2.010798e-06,5.252157e-06,2.816922e-05,0.361144,0.000052,8.335794e-06,3.813588e-05,7.125804e-06,0.000091,9.503244e-04,3.058101e-05,1.777027e-02,1.817755e-05,4.996958e-05,1.848523e-04
1,003bc2cb2,0.000002,2.193338e-03,1.165582e-07,0.000320,1.708733e-07,0.000008,2.939571e-07,1.868568e-06,1.267335e-07,4.940962e-05,1.772231e-07,1.224916e-06,0.000006,0.000007,3.943180e-07,4.120017e-06,9.573984e-01,0.000002,1.900801e-08,1.390357e-06,3.767761e-06,9.538748e-06,1.221140e-06,2.108025e-06
2,0061c037e,0.000251,1.693869e-04,2.976230e-04,0.066917,1.095120e-02,0.135221,7.886685e-04,8.922619e-03,2.388212e-04,7.535269e-04,6.147520e-04,5.069349e-03,0.000062,0.007028,1.316011e-05,1.182224e-03,9.713077e-04,0.016119,1.160421e-04,1.682614e-03,4.167228e-03,1.350930e-04,1.477856e-03,2.389252e-02
3,010eb14d3,0.963565,1.244644e-05,4.160843e-06,0.000056,5.033771e-07,0.000046,3.242720e-07,1.249164e-07,2.209541e-02,4.924443e-07,4.761642e-07,3.447671e-07,0.000003,0.000002,1.786450e-07,7.091148e-07,1.752068e-06,0.000005,4.938534e-07,7.619309e-07,2.620172e-07,4.126362e-07,9.425964e-07,7.473893e-07
4,011318064,0.000040,5.292693e-05,6.171323e-05,0.098514,2.162273e-04,0.001057,1.054170e-03,3.621771e-04,2.805390e-04,7.210280e-06,3.725988e-05,2.622039e-04,0.004005,0.000178,5.888596e-01,1.322605e-01,2.394586e-04,0.000204,5.173582e-02,1.893003e-04,1.299619e-03,2.401631e-04,2.173337e-04,3.865916e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987,ff68f3ac3,0.000003,4.796928e-08,9.119581e-04,0.038422,2.190511e-06,0.510973,4.407418e-06,6.008979e-03,5.373733e-06,2.003824e-05,5.224317e-07,6.800503e-06,0.004969,0.000058,3.386988e-08,3.259967e-01,2.373848e-07,0.000096,4.446587e-06,2.543562e-05,4.388268e-03,3.603714e-08,2.184874e-06,2.887076e-01
1988,ff973e852,0.000009,2.608355e-06,1.463524e-05,0.043611,3.405661e-06,0.001404,4.284172e-03,3.189917e-01,8.581590e-05,1.638026e-03,1.208688e-04,3.962563e-03,0.000011,0.000006,4.674890e-05,1.491550e-01,9.975363e-05,0.089006,2.358605e-05,4.462274e-04,1.820171e-03,6.044469e-06,2.954625e-03,8.638618e-04
1989,ffa5cf6d6,0.000004,6.867082e-06,6.068025e-03,0.125827,6.103484e-06,0.001370,7.000581e-05,1.009463e-03,9.694934e-05,8.071724e-06,2.297212e-05,9.789510e-05,0.000651,0.000008,5.101201e-06,3.141828e-01,3.238459e-05,0.079021,7.030174e-06,1.809574e-04,8.593509e-05,5.889536e-06,5.686501e-04,7.328827e-04
1990,ffa88cbb8,0.000040,6.662697e-04,1.410624e-04,0.509611,5.166793e-03,0.001269,5.050829e-05,1.131830e-01,1.967448e-04,7.018165e-05,3.283749e-05,1.725113e-04,0.002769,0.000662,1.224593e-04,2.300450e-04,5.824919e-02,0.000245,6.078756e-05,1.431557e-04,2.273991e-04,8.740327e-04,9.990065e-05,3.402020e-04


In [ ]:
device = torch.device("cuda")
model.eval().to(device)
preds = []

for path in tqdm(test_wav_pathes):
    y, sr = sf.read(path)
    mel_img = signal_to_mel(y, sr, config["dataset"]["params"])
#     mel_img = torch.from_numpy(mel_img).clone().to(device)
#     pred = model(mel_img)
# #         pred = torch.mean(pred['clipwise_output'], 0)
# #         pred = pred.to('cpu').detach().numpy().copy()
#     preds.append(pred)
    
    
    b_start = 0
    b_end = BATCH
    b_preds = []
    while(b_start < mel_img.shape[0]):
        if(b_end >= mel_img.shape[0]):
            mel_batch = mel_img[b_start:]
        else:
            mel_batch = mel_img[b_start:b_end]
        mel_batch = torch.from_numpy(mel_batch).clone().to(device)
        pred = model(mel_batch)
        pred = pred['framewise_output'].to('cpu').detach().numpy().copy()
        b_preds.extend(pred)
        b_start += BATCH
        b_end += BATCH
    b_preds = np.array(b_preds).reshape(-1, 24)
    pred = np.max(b_preds, axis=0)
    preds.append(pred)    
    
            
preds = np.array(preds)